In [ ]:
import pandas as pd
import numpy as np
from pandas_datareader import data, wb
from datetime import datetime
#cor plot index use is inconsistant with some other function maybe HRP?

In [ ]:
#%run /Users/davidfitzpatrick/Desktop/Codes/Python_codes/Finance_codes/tools/helpers.ipynb

#read in stock data
#tickers = ['SPY', 'SPYC']
#start = datetime(2020,1,1)
#end = datetime(2021,7,20)
#returns = import_data(tickers,start,end)

In [ ]:
from numpy.lib.stride_tricks import as_strided

def windowed_view(x, window_size):
    """Creat a 2d windowed view of a 1d array.

    `x` must be a 1d numpy array.

    `numpy.lib.stride_tricks.as_strided` is used to create the view.
    The data is not copied.

    Example:

    >>> x = np.array([1, 2, 3, 4, 5, 6])
    >>> windowed_view(x, 3)
    array([[1, 2, 3],
           [2, 3, 4],
           [3, 4, 5],
           [4, 5, 6]])
    """
    y = as_strided(x, shape=(x.size - window_size + 1, window_size),
                   strides=(x.strides[0], x.strides[0]))
    return y

def max_dd(ser):
    max2here = ser.cummax()
    dd2here = ser - max2here
    return dd2here.min()

In [ ]:
def Turbulence_indicator(returns): #reduces dataset by 15 days because covariance matrix is unstable before that
    turbulence=[]
    for i in range(15,len(returns)):
        deviation = returns.iloc[i]-returns[:i].mean() #[:i] prevents information bleed
        inverse = np.linalg.inv(returns[:i].cov())     #[:i] prevents information bleed
        turb = np.sqrt(np.matmul(np.matmul(deviation.T,inverse),deviation))
        turbulence.append(turb)
    pd.options.mode.chained_assignment = None  # default='warn'
    AllData = returns[15:]
    AllData['Turbulence']=turbulence
    pd.options.mode.chained_assignment = 'warn'  # default='warn'
    return AllData

#Turbulence indicator- I use the square root because the magnitude of the indicator is higher for daily data
#starts 15 days in for stability reasons. inverse covariance matrix is large and sometimes negative for small sample sizes

In [1]:
#%run C:\Users\avid\Desktop\Python_codes\Finance_codes\tools\helpers.ipynb
#tickers = ['TLT', 'IVV']
#start = datetime(2003,1,1)
#end = datetime(2020,7,31)
#returns = import_data(tickers,start,end, give ="R", dividend=True)

#TIP: use join when have dataframes of different lengths and just want to keep the intersection of rows


# plot dividends
    # pd.DataFrame(port["IVV_Div"]/port["IVV"]).resample('AS').sum().plot()

# annualize returns
    # returns.resample("A").apply(lambda x: ((x + 1).cumprod()-1).last("D"))

#for company info
    #si.get_quote_table("SPY", dict_result = False)

# get weekday
    # week['weekday'] = week['Date'].apply(lambda x: x.weekday())
    
#bid ask info 
    #si.get_quote_table("SPY", dict_result = False)

In [2]:
def crosscorr(leader, follower, graph_bounds, plot=False, title="Title"):

    x = np.arange(len(leader))
    y1 = follower
    y2 = leader
    npts = len(y1)

    lags = np.arange(-npts + 1, npts)
    ccov = np.correlate(y1 - y1.mean(), y2 - y2.mean(), mode='full')
    ccor = ccov / (npts * y1.std() * y2.std())

    #fig, axs = plt.subplots(nrows=2)    
    #fig.subplots_adjust(hspace=0.4)   
    #ax = axs[0]   
    #ax.plot(x, y1, 'b', label='Follower')
    #ax.plot(x, y2, 'r', label='Leader')    
    #ax.set_ylim(-.1, 1)    
    #ax.legend(loc='upper right', fontsize='small', ncol=2)

    if(plot==True):
        fig, axs = plt.subplots(nrows=1)
        ax = axs#[1]
        ax.plot(lags[graph_bounds:-graph_bounds], ccor[graph_bounds:-graph_bounds]) #npts-1, npts-1
        ax.set_ylim(-.7, .9)
        ax.set_ylabel('Correlation')
        ax.set_xlabel('Days between Leader and Follower')
        plt.title(title)
        plt.show()

    maxlag = np.argmax(ccor[:])#npts-1
    maxcorr = np.max(ccor[:])#npts-1
    print(title,": Max correlation is at lag", maxlag-npts+1,"\nCorrelation is", round(maxcorr,2)) 
    # from following https://currents.soest.hawaii.edu/ocn_data_analysis/_static/SEM_EDOF.html

In [ ]:
def beta(regressor, data, start, end): #Monthly beta
    from sklearn.linear_model import LinearRegression
    #from sklearn.base import clone
    #import sklearn
    lm = LinearRegression()
    riskfree = 'VFISX' #VFISX is a poor risk_free rate maturity currently being 3 years, but it's convenient
    #returns = import_data([regressor,riskfree]+tickers,start,end, give ="R",dividend=True) alt method
    returns_internal = import_data([regressor,riskfree],start,end, give ="R",dividend=True)
    all_data = returns_internal.join(data,how='inner')
    for col in (all_data.columns):
        all_data[col] = all_data[col]-all_data[riskfree]
    adjusted = all_data.drop(riskfree,axis=1).resample("M").apply(lambda x: ((x + 1).cumprod()-1).last("D"))
    
    coef=[]
    for col in (adjusted.columns[1:]):
        X = np.array(adjusted[regressor]).reshape(-1,1)
        y = np.array(adjusted[col]).reshape(-1,1)
        lm.fit(X,y) #use all data as training data because we arn't predicting anything
        coef.append(lm.coef_[0])
        
    coeff_df = pd.DataFrame(coef,columns=["Beta"]).set_index(adjusted.columns[1:]) 
    print(coeff_df)

In [1]:
#fixed allocation backtest with chosen rebalance schedule

def backtest(wealth, weights, col, dataf, days_between_rebalance, RebalanceOffset=0, give='R'): #cols allows users to backtest subsets of dataframe
    
    colin=[0]*len(weights)
    port=[0]*len(weights)
    worth=[wealth]

    for n in range(len(weights)):
        colin[n] = dataf.columns.get_loc(col[n])
        port[n]=wealth*weights[n]

    for n in range (len(dataf)):
        for i in range(len(weights)):
            port[i]=port[i]*(1+dataf.iloc[n,colin[i]])

        end_of_day = sum(port)
        
        worth.append(end_of_day)
        if ((n + RebalanceOffset)% days_between_rebalance==0):        #rebalance  
            
            for i in range(len(weights)): port[i] = end_of_day * weights[i]
    if (give=='R'):            
        p_rtn = pd.DataFrame(worth).pct_change()[1:].set_index(dataf.index)
        return(p_rtn)
    if (give=='D'):
        return(pd.DataFrame(worth[1:]).set_index(dataf.index))
    

#def backtest(wealth,weights,col, data, days_between_rebalance): #cols allows users to backtest subsets of dataframe
#    colin=[0]*len(weights)
#    port=[0]*len(weights)
#    worth=[wealth]
#
#    for n in range(len(weights)):
#        colin[n] = data.columns.get_loc(col[n])
#        port[n]=wealth*weights[n]
#
#    for n in range (len(data)):
#        for i in range(len(weights)):
#            port[i]=port[i]*(1+data.iloc[n,colin[i]])
#
#        end_of_day = sum(port)
#        
#        
#        if (n % days_between_rebalance==0):        #rebalance  
#            worth.append(end_of_day)
#            for i in range(len(weights)): port[i] = end_of_day * weights[i]
#                
#    p_rtn = pd.DataFrame(worth).pct_change()[1:].reset_index().drop(["index"],axis=1)
#    
#    return(p_rtn)

In [1]:
def IR_or_yield_curve(start,end,plot=True):
    
    if (plot==True):
        Interest_rates = ['^IRX','^TNX'] # '^FVX', '^TYX'
        IR = import_data(Interest_rates,start,end, give ="S", dividend=True)
        IR["yd_cve"]=IR["^TNX"]-IR["^IRX"]
        IR["yd_cve"].plot()
        plt.title("10 year - 3 month rate")
        plt.show(block=True)
    if (plot==False):
        Interest_rates = ['^IRX','^FVX','^TNX','^TYX']
        IR = import_data(Interest_rates,start,end, give ="S", dividend=True)
        return IR

In [ ]:
def IR_swap(rec_int, pay_int, rec_yr, pay_yr, rec_principal, pay_principal,length, FX_rate):
    FX_rates=[FX_rate]
    pay_cashflow=[]
    rec_cashflow=[]
    net_cashflow=[]
    present_val=[]
    for n in range (1, length+1):
        FX_rates.append(FX_rate*m.exp((pay_int-rec_int)*n))
        pay_cashflow.append(pay_principal*pay_yr)
        rec_cashflow.append(rec_principal*rec_yr)
    pay_cashflow.append(pay_principal)
    rec_cashflow.append(rec_principal)
    del FX_rates[0]
    FX_rates.append(FX_rates[n-1])
    for n in range (0,length+1):
        net_cashflow.append(rec_cashflow[n]*FX_rates[n]+pay_cashflow[n])
    for n in range(0,length):
        present_val.append(net_cashflow[n]*m.exp(-pay_int*(n+1)))
    present_val.append(net_cashflow[n+1]*m.exp(-pay_int*(n+1)))
    print(sum(present_val))
    
    #assumes term structure is flat, rates are continuously compounded

#rec_int        #interest risk free rate of currency recieving
#pay_int        #interest risk free rate of currency paying
#rec_yr         #yearly interest rate agreed upon which the company is recieving
#pay_yr         #yearly interest rate agreed upon which the company is paying
#rec_principal  #principle amount agreed in currency company is recieving
#pay_principal  #principle amount agreed in currency company is paying
#length         #years left in contract
#FX_rate        #ratio FX rate: currency paid/currency recieved

#Note: output is in units of currency which the company pays
#ex IR_swap(.04,.09,.05,.08,1200,-10,3,1/110)

#rec  yen
#pay   dollars
#present value is in currency paying

In [3]:
def geometric_return(data):
    return data.cumprod()[-1:].abs()**(1/(data[-1:].index[0]))

In [ ]:
#growth of 1 dollar invested

def dollar_growth(returns): # dates come directly from the returns provided so dates might be off one from expected
    how=np.ones([len(returns),len(returns.columns)])
    for n in range(1,len(returns)):
        for i in range(len(returns.columns)):
            how[n,i]=how[n-1,i]*(1+returns.iloc[n-1,i])
    how=pd.DataFrame(how)
    how.columns=returns.columns
    how.index=returns.index
    return how

In [2]:
#plotting the monthly time series of correlation

def cor_plot(start_year, end_year, dataset, SandP,lines_per_plot,num_plots,lower_y):
    cor = pd.DataFrame()
    month = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]
    i=0
    j=0
    for j in np.arange(start_year,end_year+1):
        for i in np.arange(1,13):
            cor[month[i-1]+str(j)] = correlation(dataset,SandP,datetime(j,i,1), datetime(j,i,28))       
    for i in np.arange(0,num_plots,1):
        print(cor[i:i+lines_per_plot].T.rolling(4).mean().plot().set_ylim(lower_y, 1))
        
def correlation(dataset,SandP,start,end):
    return dataset[start:end].corr()[SandP]

In [1]:
def import_data(tickers,start,end, give ="R", dividend=True):#read in stock data  
    SPY = data.DataReader('SPY', 'yahoo', start, end)["Close"]
    stocks = pd.DataFrame()
    stock_div = pd.DataFrame(SPY).drop("Close",axis=1)
    for tick in tickers:
        stocks[tick] = data.DataReader(tick, 'yahoo', start, end)["Close"]
        if (dividend == True): 
            stock_div[tick+' div'] = data.DataReader(tick, 'yahoo-dividends', start, end)["value"]
      
    just_stocks=stocks
    if (dividend == True):
        stocks = pd.concat([stocks,stock_div],axis=1)
        stocks = stocks.fillna(0)
        
    if(give == "R"):
        #Adding dividends to price appreciation on a daily percent return basis -useful for all analysis
        raw_returns=pd.DataFrame()
        for tick in tickers:
            if (dividend == True):
                raw_returns[tick] = stocks[tick].pct_change() + stocks[tick+' div']/stocks[tick] 
                #dividends are actually added on the day before they should be added but this error is neglegible
            else:
                raw_returns[tick] = stocks[tick].pct_change()
        returns=raw_returns[1:]
        return returns[np.isfinite(returns).all(1)]
        
    if(give == "S"):
        return just_stocks
    
    if(give=="D"):
        dividend = pd.DataFrame(np.array(stocks.iloc[:,len(just_stocks.columns):])/np.array(stocks.iloc[:,:len(just_stocks.columns)])).set_index(just_stocks.index)
        dividend.columns=just_stocks.columns
        return dividend
    
    #importing financial data from yahoo finance
#commodity_futures = ['GC=F', 'SI=F', 'CL=F']
#cryptocurrencies = ['BTC-USD', 'ETH-USD', 'XRP-USD']
#currencies = ['EURUSD=X', 'JPY=X', 'GBPUSD=X']
#mutual_funds = ['PRLAX', 'QASGX', 'HISFX']
#us_treasuries = ['^TNX', '^IRX', '^TYX']

In [ ]:
def SMA_Backtest(SMA_len, return_len, data, initial):
    timing = initial
    hold = 'cash'
    acct_value = []
    trades=[]
    
    for n in range (SMA_len, len(data)-return_len-np.mod(len(data)-SMA_len, return_len),return_len): 
        #the stopping point maximises the loops us of dtaset without falling outside of it.
        price = data.iloc[n]
        next_price = data.iloc[n+return_len]
        SMA = data.iloc[n-SMA_len:n].mean()
        acct_value.append(timing)
        if (price >= SMA) & (hold =='stock'): #the first case for when the loop checks for a price signal
            timing = next_price/price*timing
        elif (price >= SMA) & (hold =='cash'):#the second case for when the loop checks for a price signal
            hold = 'stock'
            open_position = price
            timing = next_price/price*timing
        elif (price < SMA) & (hold =='stock'):
            #the third and final case for when the loop checks for a price signal else nothing happens.
            hold ='cash'
            close_position = price
            if close_position > open_position:
                trades.append(1)
            else: 
                trades.append(0)
    return(timing,acct_value,trades)

#The SMA function,includes current day included SMA. basic form is to compare price to SMA of length set by user.
#This function looks for another price signal after a number of days equal to the return length

In [1]:
# total daily contract volume

def daily_call_volume(ticker):
    day_vol = 0
    exp_dates = options.get_expiration_dates(ticker)
    for dates in exp_dates:
        calls = options.get_calls(ticker,dates)["Volume"].values
        calls = np.where(calls == '-', 0, calls)
        day_vol += calls.astype('int').sum()

In [ ]:
def conditional_relative_performance(df,select_intervals=[.02,.01,.005,0,-.005,-.01,-.02]):
    #select_intervals = [.02,.01,0,-.01,-.02] #highest to lowest
    outperformance = np.zeros(len(select_intervals)+1)
    probabilities = np.zeros(len(select_intervals)+1)
    outperformance[0] = df[df['SPY']>=select_intervals[0]].mean().diff()[1]
    probabilities[0] = len(df[df['SPY']>=select_intervals[0]])/len(df)

    for i in range(1,len(select_intervals)):
        outperformance[i] = df[(df['SPY']<select_intervals[i-1]) & (df['SPY']>select_intervals[i])].mean().diff()[1]
        probabilities[i] = len(df[(df['SPY']<select_intervals[i-1]) & (df['SPY']>select_intervals[i])])/len(df)
    
    outperformance[-1] = df[df['SPY']<=select_intervals[-1]].mean().diff()[1]
    probabilities[-1] = len(df[df['SPY']<=select_intervals[-1]])/len(df)


    print('Given SPY moves more than',select_intervals[0], 'SPYC outperformance is', np.round(outperformance[0],4), '    Probability:', np.round(probabilities[0],4))
    for i in range(len(outperformance)-2):
        print('{0: <16}'.format('Given SPY moves'), round(select_intervals[i+1],4),'to',round(select_intervals[i],4),'SPYC outperformance is',np.round(outperformance[i+1],4),'Probability:', np.round(probabilities[i+1],3) )
    print('Given SPY moves below ',select_intervals[-1], 'SPYC outperformance is', np.round(outperformance[-1],4),'    Probability:', np.round(probabilities[-1],4))  
    print('Expected outperformance: ', np.dot(probabilities,outperformance))